# P3. Exploration SQL

Le répertoire `/db` contient une base de données SQLite3 `olympics.sqbpro` reprenant les résultats des épreuves des jeux d'été pour une période donnée (cf P1 & P2).

On s'intéresse dans le cadre de cette partie P3 à l'exploration via SQL des résultats des JO d'été (table : `summer`).

---

### Question 1: Qu'est ce que SQL ?

Le SQL **(Structured Query Language)** est un langage permettant de communiquer avec une base de données.
Ce langage informatique est notamment très utilisé par les développeurs web pour communiquer avec les données d’un site web. Il est possible de créer ou supprimer des objets dans la base de données, mais aussi manipuler des bases de données en effectuant des requêtes qui permettent d'analyser les données de celle-ci.

Ce langage est structuré, c’est-à-dire que la syntaxe est toujours la même et respecte des normes très précises.

### Question 2: Qu'est ce que SQLite 3 ? Avantages et/ou Inconvénients ?

**SQLite** est un système de base de données simplifié de MySQL. SQLite3 est la 3ème version qui apporte de meilleures
performances que les versions précédentes, telles que l'auto increment, un stockage plus efficace et un support des clés étrangères.
Les principaux avantages de SQLite est qu'il est plus simple d'utilisation, fonctionne sans serveur, et possède la majeure partie des fonctions classiques.

Il manque en revanche certaines fonctions qui peuvent s'avérer utiles dans certains cas (ex : ALTER TABLE, COUNT (DISTINCT X). Il ne reconnaît par ailleurs ni les transactions imbriquées, ni les déclencheurs, et ne permet pas la réplication. Il est donc réservé à des besoins basiques.

### Question 3: Donner des exemples d'autres formats de fichier pouvant être utilisé pour fournir des données similaires 

.xls, json

### Question 4: Quel est le type (texte ou binaire) du fichier base de données SQLite3?

Le fichier `olympics.sqbpro` est un fichier binaire.

### Question 5: Donner les caractéristiques de la table `summer` ? 

Cette table donne tous les médaillés des Jeux olympiques d'été, en détaillant pour chaque médaille remportée :

*Année de l'édition

*Ville de l'édition

*Catégorie du sport

*Sport pratiqué

*Nom de l'athlète

*Nationalité de l'athlète

*Genre de l'athlète

*Nom de l'épreuve

*Médaille (or, argent, bronze)

---
### Question 6: Combien d'enregistrements possède la table `summer` ?

SELECT count(*)

FROM summer

![Question6](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Question_6.png)

### Question 7: Quelle période des JO couvre ce fichier de résultats ?

SELECT min(Year), max(Year)

FROM summer

![Question7](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Question_7.png)

### Question 8: Indiquer les années et villes des olympiades ? 

SELECT DISTINCT Year, City

FROM summer

![Question8](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Question_8.png)

### Questions Google Drive: 

**Q3. En 2012 combien de médailles le Japon a t-il gagné ?**

SELECT count(*)

FROM (

SELECT *

FROM summer

WHERE Year = 2012 AND Country = "JPN"

GROUP BY Sport, Discipline, Gender, Event

)

![Q.3](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q3.png)

**Q17. Entre 1984 et 2012 combien de médailles a gagné la France ?**

SELECT count(*)

FROM(

SELECT *

FROM summer

WHERE Year >= 1984 AND Year <= 2012 AND Country = "FRA"

GROUP BY Year, Sport, Discipline, Gender, Event

)

![Q17](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q17.png)

**Q10. Combien de joueuses comptait la sélection brésilienne de volley-ball médaillée d’or aux J.O de 2012 ?**

SELECT count(*)

FROM summer

WHERE Country = "BRA" AND Discipline = "Volleyball" AND Gender = "Women" AND Year = 2012

![Q10](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q10.png)

**Q13. Est-ce que la France à déjà gagné une médaille d’or au cricket ?**

SELECT count(*)

FROM summer

WHERE Country = "FRA" AND Discipline = "Cricket" AND Medal = "Gold"

![Q13](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q13.png)

**Q12. Quel est l’athlète qui a remporté le plus de médailles d’or ?**

SELECT Athlete

FROM summer

WHERE Medal = "Gold"

GROUP BY Athlete

ORDER BY COUNT(*) DESC

LIMIT 1

![Q12](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q12.png)

**Q5. Quel pays a remporté le plus de médailles d’or ?**

SELECT Country, count(*)

FROM(

SELECT *

FROM summer

WHERE Medal = "Gold"

GROUP BY Year, Sport, Discipline, Gender, Event

ORDER BY Count(*) DESC

)

GROUP BY Country

ORDER BY count(*) DESC

LIMIT 1

![Q5](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q5.png)

**Q15. Le nombre de médailles d’or par pays en 2012**

SELECT Country, count(*)

FROM(

SELECT *, count(*)

FROM summer

WHERE Medal = "Gold" AND Year = 2012

GROUP BY Sport, Discipline, Event, Gender

)

GROUP BY Country

ORDER BY count(*) DESC

![Q15](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q15.png)

**Q1. Combien la France a remporté de médailles d’or aux JO de 1984 ?**

*#requête imbriquée#*

SELECT Count(*)

FROM(

SELECT Athlete, Sport, Discipline, Event, Gender, count(*)

FROM summer

WHERE Year = 1984 AND Country = "FRA"  AND Medal = "Gold"

GROUP BY Sport, Discipline, Event, Gender

)

![Q.1](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q1.png)

**Q11. Combien de femmes françaises ont ont remporté des médailles en 2012, quelle sont leurs médailles ?**

SELECT count(DISTINCT Athlete)

FROM summer

WHERE Gender = "Women" AND Country = "FRA" AND Year = "2012"

![Q11_1](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q11_1.png)

SELECT Medal, count(*)

FROM summer

WHERE Gender = "Women" AND Country = "FRA" AND Year = "2012"

GROUP BY Medal

![Q11_2](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q11_2.png)

**Q9. Combien de fois Londres a accueilli les J.O ?**

SELECT count(DISTINCT Year)

FROM summer

WHERE City = "London"

![Q9](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q9.png)

**Q6. Combien de villes différentes ont accueilli les JO d’été entre 1896 et 2012 ?**

SELECT count(DISTINCT City)

FROM summer

WHERE Year >=1896 AND Year <= 2012

![Q6](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q6.png)

**Q16. Combien de femmes ont remporté des médailles ?**

SELECT count(DISTINCT Athlete)

FROM summer

WHERE Gender = "Women"

![Q16](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q16.png)

**Q7. Combien de femmes ont obtenu  une médaille d’or ?**

SELECT count(DISTINCT Athlete)

FROM summer

WHERE Gender = "Women" AND Medal = "Gold"

![Q7](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q7.png)

**Q2. Quel pays a le plus d’athlètes ?**

SELECT Country, count(DISTINCT Athlete)

FROM summer

GROUP BY Country

ORDER BY count(DISTINCT Athlete) DESC

LIMIT 1

![Q2](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q2.png)

**Q8. Combien de sports différents sont représentés par année ?**

SELECT Year, count(DISTINCT Sport)

FROM summer

GROUP BY Year

![Q.8](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q8.png)

**Q4. Dans quels différents sports les Grecs ont-ils gagné des médailles ?**

SELECT Sport, count(Medal)

FROM summer

WHERE Country = "GRE"

GROUP BY Sport

ORDER BY count(Medal) DESC

![Q4](C:\Users\levog\Simplon\jeux-olympiques\SQLJO\Q4.png)

--- 

Fin